## Install

實作部落格說明：
https://reurl.cc/E2x1jk

In [ ]:
!pip install git+https://github.com/finlab-python/finlab.git

  Cloning https://github.com/finlab-python/finlab.git to /tmp/pip-req-build-it9sjwbe
  Running command git clone -q https://github.com/finlab-python/finlab.git /tmp/pip-req-build-it9sjwbe
     |████████████████████████████████| 1.8MB 10.2MB/s 
     |████████████████████████████████| 13.2MB 294kB/s 
     |████████████████████████████████| 5.0MB 30.0MB/s 
     |████████████████████████████████| 245kB 47.3MB/s 
     |████████████████████████████████| 18.1MB 172kB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 5.7MB 14.6MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 215kB 52.9MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 1.0MB 31.8MB/s 
     |████████████████████████████████| 3.5MB 28.0MB/s 
     |█████████████████████████████

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import requests
import finlab
from finlab import data
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [ ]:
finlab.set_token('vPrVSU1klPn98i5OBK3HQRjQBhR9odXvB6dqSOsU/GvjjkNSgQ2Q0vr7Y/kSI8Wa')

In [ ]:
plan_vol=data.get('treasury_stock:預定買回股數')
# low=data.get('treasury_stock:買回價格區間-最低')
# high=data.get('treasury_stock:買回價格區間-最高')
# equality_ratio=data.get('treasury_stock:本次買回股數佔公司已發行股份總數比例(%)')
# vol=data.get('treasury_stock:本次已買回股數')
# avg_cost=data.get('treasury_stock:本次平均每股買回價格')
# close=data.get('price:收盤價')

## 每月庫藏股家數統計

次月1日得知上月統計家數，
取標準差抓庫藏股家數特多的月份為訊號

In [ ]:
plan_vol=data.get('treasury_stock:預定買回股數')
plan_vol=plan_vol.unstack().dropna()
plan_vol=plan_vol.reset_index()
plan_vol.columns=['stock_id','date','plan_vol']

# 次月1日得知上月統計家數，取標準差抓庫藏股家數特多的月份為訊號
plan_vol['date']=plan_vol['date']+ pd.DateOffset(months=1)
plan_vol['year_month']=plan_vol['date'].apply(lambda s:datetime(s.year,s.month,1))
count_num=plan_vol.groupby(['year_month']).count().iloc[:,0].to_frame()
count_num['signal'] = ['crimson' if x > int(count_num.mean()+1.5*np.std(count_num).values) else 'darkcyan' for x in count_num.iloc[:,0]]
count_num=count_num.reset_index()
count_num=count_num.rename(columns={'stock_id':'count'})
count_num


,year_month,count,signal
0,2000-09-01,14,darkcyan
1,2000-10-01,30,darkcyan
2,2000-11-01,66,darkcyan
3,2000-12-01,66,darkcyan
4,2001-01-01,41,darkcyan
...,...,...,...
244,2021-02-01,10,darkcyan
245,2021-03-01,8,darkcyan
246,2021-04-01,11,darkcyan
247,2021-05-01,1,darkcyan


In [ ]:
benchmark=data.get('benchmark_return:發行量加權股價報酬指數')
benchmark = benchmark.reindex(count_num['year_month'], method='bfill')
benchmark=benchmark.reset_index()
benchmark

,year_month,發行量加權股價報酬指數
0,2000-09-01,4524.92
1,2000-10-01,4524.92
2,2000-11-01,4524.92
3,2000-12-01,4524.92
4,2001-01-01,4524.92
...,...,...
244,2021-02-01,29749.49
245,2021-03-01,30785.79
246,2021-04-01,32036.26
247,2021-05-01,33311.88


### Plot

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=count_num['year_month'], y=count_num['count'],marker_color=count_num['signal'], name="count"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=benchmark['year_month'], y=benchmark['發行量加權股價報酬指數'],line = dict(color = '#2C191B'), name="發行量加權股價報酬指數"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Treasury Stock Month Static Signal"
)

# Set x-axis title
fig.update_xaxes(title_text="year-month",
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1,
                              label="1y",
                              step="year",
                              stepmode="backward"),
                         dict(count=3,
                              label="3y",
                              step="year",
                              stepmode="backward"),
                         dict(count=5,
                              label="5y",
                              step="year",
                              stepmode="backward"),
                         dict(count=10,
                              label="10y",
                              step="year",
                              stepmode="backward"),
                         dict(step="all")
                     ])
                 ),
                 rangeslider=dict(
                     visible=True
                 ),
                 type="date")



# Set y-axes titles
fig.update_yaxes(title_text="<b>count</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>benchmark</b>", secondary_y=True)

fig.show()

## Crawler

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import requests


def combine_index(df, n1, n2):
    """將dataframe df中的股票代號與股票名稱合併

    Keyword arguments:

    Args:
        df (pandas.DataFrame): 此dataframe含有column n1, n2
        n1 (str): 股票代號
        n2 (str): 股票名稱

    Returns:
        df (pandas.DataFrame): 此dataframe的index為「股票代號+股票名稱」
    """

    return df.set_index(df[n1].str.replace(' ', '') + \
                        ' ' + df[n2].str.replace(' ', '')).drop([n1, n2], axis=1)


def crawl_treasury_stock(days=200):
    now =datetime.now().strftime('%Y%m%d')
    end = (datetime.now() - timedelta(days)).strftime('%Y%m%d')
    date_now = str(int(now[:4]) - 1911) + now[4:]
    date_end = str(int(end[:4]) - 1911) + end[4:]

    data = []
    for market_name in ["sii", "otc"]:
        url = 'https://mops.twse.com.tw/mops/web/t35sc09'
        form_data = {
            "encodeURIComponent": "1",
            "step": "1",
            "firstin": "1",
            "off": "1",
            "TYPEK": market_name,
            "d1": date_end,
            "d2": date_now,
            "RD": "1",
        }

        res = requests.post(url, form_data)
        page_count = len(pd.read_html(res.text))
        page_start = 11
        page_end = page_count - 2
        df = pd.concat([pd.read_html(res.text)[n] for n in range(page_start, page_end)])
        data.append(df)

    df_all = pd.concat(data)
    df_all = df_all.astype(str)
    df_all.columns = ['序號', 'stock_id', '公司名稱', 'date', '買回目的', '買回股份總金額上限',
                      '預定買回股數', '買回價格區間-最低', '買回價格區間-最高', '預定買回期間-起', '預定買回期間-迄', '是否執行完畢',
                      '買回達一定標準資料', '本次已買回股數', '本次執行完畢已註銷或轉讓股數',
                      '本次已買回股數佔預定買回股數比例(%)', '本次已買回總金額',
                      '本次平均每股買回價格', '本次買回股數佔公司已發行股份總數比例(%)', '本次未執行完畢之原因']

    df_all = df_all.drop(columns=['序號', '買回達一定標準資料'])

    def date_process(columns_name):
        df_all[columns_name] = df_all[columns_name].apply(lambda s: s.replace('/', '-')).apply(
            lambda s: datetime.strptime(str(int(s[:s.index('-')]) + 1911) + s[s.index('-'):], '%Y-%m-%d'))

    for i in ['date', '預定買回期間-起', '預定買回期間-迄']:
        date_process(i)
    float_list = ['買回股份總金額上限', '預定買回股數', '買回價格區間-最低', '買回價格區間-最高',
                  '本次已買回股數', '本次執行完畢已註銷或轉讓股數', '本次已買回股數佔預定買回股數比例(%)',
                  '本次已買回總金額', '本次平均每股買回價格', '本次買回股數佔公司已發行股份總數比例(%)']
    df_all[float_list] = df_all[float_list].apply(lambda s: pd.to_numeric(s, errors='coerce')).fillna(0)
    df_all['買回目的'] = ['轉讓股份予員工' if i == "1" else '股權轉換' if i == "2" else '維護公司信用及股東權益' for i in df_all['買回目的']]
    df_all = combine_index(df_all, "stock_id", "公司名稱")
    df_all.index.name = 'stock_id'
    df_all = df_all.reset_index()
    df_all = df_all.set_index(['stock_id', 'date'])
    sort_multi=df_all.index.sortlevel(1,sort_remaining=True)
    treasury_stock=df_all.loc[sort_multi[0]]
    return treasury_stock

In [ ]:
treasury_stock=crawl_treasury_stock()
treasury_stock

,,買回目的,買回股份總金額上限,預定買回股數,買回價格區間-最低,買回價格區間-最高,預定買回期間-起,預定買回期間-迄,是否執行完畢,本次已買回股數,本次執行完畢已註銷或轉讓股數,本次已買回股數佔預定買回股數比例(%),本次已買回總金額,本次平均每股買回價格,本次買回股數佔公司已發行股份總數比例(%),本次未執行完畢之原因
stock_id,date,,,,,,,,,,,,,,,
6446 藥華藥,2020-10-28,轉讓股份予員工,715459354,3200000,57.00,126.00,2020-10-29,2020-12-27,Y,2935000.0,0.0,91.72,2.573847e+08,87.69,1.12,因本公司視股價變化及成交量狀況，採取適度分批買回策略，故未執行完畢。
4760 勤凱科技,2020-11-02,轉讓股份予員工,255851356,500000,52.00,80.00,2020-11-03,2020-12-31,Y,40000.0,0.0,8.00,3.072062e+06,76.80,0.13,兼顧市場機制及資金運用效益，本公司視股價變化、成交量狀況及考量員工認購意願，故未全數執行完畢。
8104 錸寶,2020-11-03,轉讓股份予員工,1193737305,1000000,33.00,70.00,2020-11-04,2021-01-03,Y,1000000.0,0.0,100.00,5.148674e+07,51.49,1.48,nan
2340 光磊,2020-11-06,轉讓股份予員工,3103738614,7500000,16.70,35.20,2020-11-09,2021-01-08,Y,4294000.0,0.0,57.25,1.092507e+08,25.44,1.13,為維護整體股東權益及兼顧市場機制，視股價變化於價格區間內採分批買回策略，故未執行完畢。
6161 捷波,2020-11-09,維護公司信用及股東權益,486938914,2000000,15.00,20.00,2020-11-10,2021-01-08,Y,2000000.0,2000000.0,100.00,3.600722e+07,18.00,2.53,nan
2460 建通,2020-11-11,轉讓股份予員工,550384476,1500000,13.00,17.00,2020-11-12,2021-01-11,Y,1186000.0,0.0,79.07,1.744186e+07,14.71,0.70,為兼顧市場機制，本公司視股價變化及成交量狀況，採分批買回策略，且買回期間部分交易日市場價格高...
1455 集盛,2020-11-13,維護公司信用及股東權益,518304754,30000000,6.04,12.52,2020-11-16,2021-01-12,Y,18313000.0,18313000.0,61.04,1.705691e+08,9.31,3.33,本公司為兼顧市場交易機制及維護整體股東權益，視股價變化及成交量執行分批買回庫藏股，考量買回期...
2611 志信,2020-11-13,維護公司信用及股東權益,623546046,2000000,4.96,8.00,2020-11-16,2021-01-15,Y,1174000.0,1174000.0,58.70,8.664551e+06,7.38,0.68,股價超過董事會授權範圍。
1591 駿吉-KY,2020-12-04,轉讓股份予員工,89629361,500000,16.00,26.00,2020-12-07,2021-02-06,Y,500000.0,0.0,100.00,9.761040e+06,19.52,1.51,nan


### search

In [ ]:
start='2021-5-1'
end='2021-5-30'

date=treasury_stock.index.get_level_values(1)
if start:
    treasury_stock=treasury_stock.loc[date>=start]
if end:
    treasury_stock=treasury_stock.loc[date<=end]
treasury_stock 

,,買回目的,買回股份總金額上限,預定買回股數,買回價格區間-最低,買回價格區間-最高,預定買回期間-起,預定買回期間-迄,是否執行完畢,本次已買回股數,本次執行完畢已註銷或轉讓股數,本次已買回股數佔預定買回股數比例(%),本次已買回總金額,本次平均每股買回價格,本次買回股數佔公司已發行股份總數比例(%),本次未執行完畢之原因
stock_id,date,,,,,,,,,,,,,,,
2492 華新科,2021-05-03,轉讓股份予員工,28265698000,400000,230.00,252.00,2021-05-04,2021-07-02,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
6640 均華,2021-05-04,轉讓股份予員工,536603727,1000000,52.00,114.00,2021-05-05,2021-07-04,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
6269 台郡,2021-05-05,維護公司信用及股東權益,19047942934,15000000,95.00,130.00,2021-05-06,2021-07-05,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
8050 廣積,2021-05-07,轉讓股份予員工,2918556230,3000000,35.00,55.00,2021-05-10,2021-07-09,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
3661 世芯-KY,2021-05-10,維護公司信用及股東權益,2540466382,1000000,280.00,720.00,2021-05-11,2021-07-09,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
1586 和勤,2021-05-13,轉讓股份予員工,768429515,3000000,28.56,69.43,2021-05-14,2021-07-13,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
2327 國巨,2021-05-13,轉讓股份予員工,64761752830,4000000,409.50,655.00,2021-05-17,2021-07-16,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
2365 昆盈,2021-05-13,維護公司信用及股東權益,982321619,5000000,10.00,15.00,2021-05-14,2021-07-12,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
3594 磐儀,2021-05-14,轉讓股份予員工,781786100,1000000,20.00,30.00,2021-05-17,2021-07-16,N,0.0,0.0,0.0,0.0,0.0,0.0,nan
